In [1]:
from pathlib import Path
import os
import pandas as pd
import pyperclip
import subprocess
from reload_recursive import reload_recursive
import helpers

import mri_data

In [2]:
reload_recursive(mri_data)
from mri_data import file_manager as fm
from mri_data import utils

pd.options.mode.copy_on_write = True

In [3]:
drive_root = fm.get_drive_root()
dataroot = drive_root / "3Tpioneer_bids"
labelroot = drive_root / "srs-9/3Tpioneer_bids_predictions"
data_dir = Path("/home/srs-9/Projects/ms_mri/analysis/paper1/data0")
df = pd.read_csv(data_dir / "manual_labels.csv")
df.set_index("subid", inplace=True)

df_full = pd.read_csv(data_dir / "t1_2_data_full.csv", index_col="subid")
df_full = helpers.set_dz_type5(df_full)

In [4]:
df_full['sub-ses']

subid
1001    sub-ms1001/ses-20170215
1002    sub-ms1002/ses-20200521
1003    sub-ms1003/ses-20170329
1004    sub-ms1004/ses-20190906
1005    sub-ms1005/ses-20200708
                 ...           
3020    sub-ms3020/ses-20170307
3008    sub-ms3008/ses-20180827
3014    sub-ms3014/ses-20161214
3018    sub-ms3018/ses-20170530
3012    sub-ms3012/ses-20200606
Name: sub-ses, Length: 575, dtype: object

In [5]:
df_check = df_full.sort_values(by="pineal_volume")[['pineal_volume', 'ms_type']]
df_check

,pineal_volume,ms_type
subid,,
1246,1.024004,RRMS
2027,8.703979,SPMS
1541,15.872060,OIND
1487,16.384026,RRMS
1155,28.159927,SPMS
...,...,...
3020,NaN,NaN
3008,NaN,NaN
3014,NaN,NaN


In [7]:
# 2039
sub = 1548
subroot = dataroot / df_full.loc[sub, 'sub-ses']

t1 = subroot / "t1.nii.gz"
flair = subroot / "flair.nii.gz"
t1_gd = subroot / "t1_gd.nii.gz"

pin1 = subroot / "pineal-SRS.nii.gz"
pin2 = subroot / "pineal-SRS_T1.nii.gz"
pin_inf = drive_root / df_full.loc[sub, 'label_folder']

images = []
for im in [t1, flair, t1_gd]:
    if im.exists():
        images.append(str(im))

labels = []
for lab in [pin_inf, pin1, pin2]:
    if lab.exists():
        labels.append(str(lab))

print("Images: \n{}".format("\n".join(images)))
print("Labels: \n{}".format("\n".join(labels)))

cmd = utils.open_itksnap_workspace_cmd(images, labels, win=True)
pyperclip.copy(cmd)
# subprocess.Popen(cmd.split(" "))

Images: 
/mnt/h/3Tpioneer_bids/sub-ms1548/ses-20210628/t1.nii.gz
/mnt/h/3Tpioneer_bids/sub-ms1548/ses-20210628/flair.nii.gz
/mnt/h/3Tpioneer_bids/sub-ms1548/ses-20210628/t1_gd.nii.gz
Labels: 
/mnt/h/3Tpioneer_bids/sub-ms1548/ses-20210628/choroid_t1_flair-CH.pineal-SRS.pituitary-CH.nii.gz
/mnt/h/3Tpioneer_bids/sub-ms1548/ses-20210628/pineal-SRS.nii.gz
/mnt/h/3Tpioneer_bids/sub-ms1548/ses-20210628/pineal-SRS_T1.nii.gz


In [5]:
for i, row in df.iterrows():
    scanroot = dataroot / str(row['sub-ses']) 
    images = []
    for im in ["t1.nii.gz", "t1_gd.nii.gz", "flair.nii.gz"]:
        if (scanroot / im).is_file():
            images.append(str(scanroot / im))
    if len(images) == 0:
        continue
    labels = []
    for lab in ["pineal-SRS.nii.gz", "pineal-SRS_T1.nii.gz"]:
        if (scanroot / lab).is_file():
            labels.append(str(scanroot / lab))
    
    cmd = utils.open_itksnap_workspace_cmd(images, labels, win=False)
    df.loc[i, "cmd"] = cmd

In [ ]:
sub = 1010
cmd = df.loc[sub, "cmd"]
print(cmd)
pyperclip.copy(cmd)
# subprocess.Popen(cmd.split(" "))

In [9]:
for i, row in df_full.iterrows():
    scanroot = dataroot / str(row['sub-ses']) 
    images = []
    for im in ["t1.nii.gz", "t1_gd.nii.gz", "flair.nii.gz"]:
        if (scanroot / im).is_file():
            images.append(str(scanroot / im))
    if len(images) == 0:
        continue
    labels = []
    for lab in ["pineal-SRS.nii.gz", "pineal-SRS_T1.nii.gz"]:
        if (scanroot / lab).is_file():
            labels.append(str(scanroot / lab))
    
    cmd = utils.open_itksnap_workspace_cmd(images, labels, win=True)
    df_full.loc[i, "cmd"] = cmd

In [24]:
man_pineal_df = pd.DataFrame(columns=df_full.columns)
for i, row in df_full.iterrows():
    scanroot = dataroot / str(row['sub-ses']) 
    if (scanroot / "pineal-SRS.nii.gz").is_file() or (scanroot / "pineal-SRS_T1.nii.gz").is_file():
        man_pineal_df.loc[i, :] = row

In [ ]:
man_pineal_df['dz_type5']

In [11]:
sub = 1066
cmd = df_full.loc[sub, "cmd"]
print(cmd)
pyperclip.copy(cmd)
# subprocess.Popen(cmd.split(" "))

itksnap -g H:/3Tpioneer_bids/sub-ms1066/ses-20180828/t1.nii.gz -o H:/3Tpioneer_bids/sub-ms1066/ses-20180828/t1_gd.nii.gz H:/3Tpioneer_bids/sub-ms1066/ses-20180828/flair.nii.gz -s H:/3Tpioneer_bids/sub-ms1066/ses-20180828/pineal-SRS_T1.nii.gz


In [23]:
df_check = df_full[["pineal_volume", "dz_type5"]]
df_check.sort_values(by="pineal_volume", inplace=True)

In [25]:
import csv

file = "/mnt/h/srs-9/hips-thomas/sub1001-20170215/left/nucleiVols.txt"
with open(file, 'r') as f:
    reader = csv.reader(f, delimiter=" ")
    vols = {row[0]: float(row[1]) for row in reader}

In [26]:
vols['7-VPL']

269.82724